In [1]:
using Pkg 
Pkg.add("NamedArrays")
Pkg.build("NamedArrays")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`


In [2]:
using JuMP, Cbc

# Days and ushers are now directly mapped to integers for easier indexing
days = 1:7 # 1=Sunday, 2=Monday, ..., 7=Saturday
ushers_shift = 1:4 # 1=A, 2=B, 3=C, 4=D

coverage = 
[1 0 0 0 0 1 1;
 1 1 1 1 1 0 0;
 0 0 1 1 1 1 0;
 1 0 0 0 1 1 1]

ushers_req = [30,8,15,20,25,30,50]

shift_cost = [250,300,200,300]

m = Model()

# number of ushers
@variable(m, x[ushers_shift] >= 0)

#objective is to minimize salary costs 
@objective(m, Min, sum(shift_cost[j]*x[j] for j in ushers_shift))

@constraint(m, days_coverage[i in days],
    sum(coverage[j,i]*x[j] for j in ushers_shift)
        >= ushers_req[i])

set_optimizer(m, Cbc.Optimizer)
optimize!(m)

println("Optimal cost: \$", objective_value(m))


#for j in ushers 
 #   println(j, ": ", value(x[j]))
#end

println("Number of ushers starting work each shift:")
for j in ushers_shift
    println("Type ", ['A','B','C','D'][j], ": ", value(x[j]))
end


Optimal cost: $17550.0
Number of ushers starting work each shift:
Type A: 45.0
Type B: 8.0
Type C: 12.0
Type D: 5.0
Presolve 5 (-2) rows, 4 (0) columns and 13 (-3) elements
0  Obj 2400 Primal inf 131 (5)
3  Obj 17550
Optimal - objective value 17550
After Postsolve, objective 17550, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 17550 - 3 iterations time 0.002, Presolve 0.00
